In [197]:
import googlemaps
import folium
import pandas as pd
import numpy as np
import heapq
from haversine import haversine #거리구하기
from sklearn.preprocessing import minmax_scale #정규화


##data 불러오기
total_df = pd.read_csv("all_df.csv", encoding = "EUC-KR" )
df = total_df.rename(index=str, columns={"Unnamed: 0" : "ID"})
df = df.drop(columns = ['경유여부','거리'])
print(len(df))

#중복값 검사해서 drop
# a  = df[ df['총밝기'] > 120]
# b = list(set(a['명칭'].values))
# df  = df[ ~df['명칭'].isin(b) ]
l_max = 125
df  = df[ df['총밝기'] < l_max]

print(len(df))

##출발지, 도착지
sp = (37.506059, 127.036863) #출발지 37.506059, 127.036863
ep = (37.509122, 127.043816) #도착지 37.509122, 127.043816


df = df.append(df.from_dict({'ID' : -1,'point_ID':'Point_SP', '위도':sp[0], '경도':sp[1], '밝기':-l_max
                             , '분류':'출발지', '명칭':'출발지', '총밝기' : -l_max }, orient = 'index').T)

df = df.append(df.from_dict({'ID' : len(df), 'point_ID':'Point_EP', '위도':ep[0], '경도':ep[1], '밝기':l_max
                             , '분류':'도착지', '명칭':'도착지', '총밝기' : l_max}, orient = 'index').T)

df = df.sort_values(by='ID').reset_index(drop=True)
df
# #df['ID'] = df.index.valuesd
# df = df.set_index('ID')


2090
2004


,ID,point_ID,경도,명칭,밝기,분류,위도,총밝기
0,-1,Point_SP,127.037,출발지,-125,출발지,37.5061,-125
1,5,Point_5,127.031,역삼1동832-25,5,보안등,37.4928,10
2,6,Point_6,127.032,역삼1동832-27,5,보안등,37.4926,50
3,7,Point_7,127.032,역삼1동832-28,5,보안등,37.4925,50
4,8,Point_8,127.032,역삼1동832-28,5,보안등,37.4925,50
5,9,Point_9,127.032,역삼1동832-29,5,보안등,37.4923,75
6,12,Point_12,127.032,역삼1동837-33,5,보안등,37.4916,65
7,13,Point_13,127.032,역삼1동837-15,5,보안등,37.4919,40
8,14,Point_14,127.032,역삼1동837-16,5,보안등,37.492,65
9,15,Point_15,127.033,역삼1동836,5,보안등,37.492,5


In [198]:
#총 거리
d_m = haversine(sp, ep, unit='m') # in meters
d_km = haversine(sp, ep,  unit='m')  # in kilometers

#block단위 
# 경유지가 총 3개까지만 가능하니까 3개의 반경으로 나누자. 700 이상인 경우는 api다시 받기 (추후에)
dvd = d_m / 4
print( "단위거리(dvd) : ",  dvd, ", 총거리(d_m) : " , d_m)

단위거리(dvd) :  175.38377731330854 , 총거리(d_m) :  701.5351092532342


## 함수정리
1. get_points_in_range : 중심점에서 범위 내 포함되는 점들 검색
2. get_gf : G, H 구하기. return 은 F
3. get_scaled_f : F scaling

In [204]:
# 초기화
point = sp
sel_pointID = 'Point_SP'
radius = 100 #반경
closed_list = []
closed_list_id = []
closed_list_seq = []
#goodbye = pd.DataFrame(columns = ['위도', '경도'] )
#반경 내 포함 points 검색
def get_points_in_range(df, point):
    rad = radius
    
    while True :
        res = df [ df.apply( lambda x : haversine(point, [x['위도'], x['경도']] ,  unit='m') , axis = 1  ) < rad ]
        res = res[ ~res['point_ID'].isin(closed_list_id) ] # 1-2 . 이미 선택했던 point들 제외   
        if (len(res) == 0  ) : 
            rad += 10
        else : 
            break
    return res

#가중치 구하기
#1. F구하기 
def get_f( df ):
    g = np.array(df.apply(lambda x : haversine(sp, [x['위도'], x['경도']] ,  unit='m') , axis = 1 )).reshape(-1,1)
    h = np.array(df.apply(lambda x : haversine(ep, [x['위도'], x['경도']] ,  unit='m') , axis = 1 )).reshape(-1,1)  
    return g+h

#2. Scaled_F : 밝기 0~100 에 맞추기
def get_scaled_f(df):
    X_MinMax_scaled = l_max - minmax_scale(df['F'], axis=0, copy=True, feature_range =(0,l_max))
    return X_MinMax_scaled

#3. 밝기 이용한 가중치 추가 W(weight = 4F(거리) + 6L(밝기))
def get_w(df) : 
    w = np.array(df.apply(lambda x : (x['총밝기'] * 45 ) + ( x['scaled_F'] * 100) , axis = 1)).reshape(-1,1)
    w = minmax_scale(w, copy=True, feature_range =(0,l_max))
    return w

In [205]:
#1. 해당 ID Close List에 추가
n = 0
heapq.heappush(closed_list_id, 'Point_SP' )
heapq.heappush(closed_list_seq, n )
heapq.heappush(closed_list, point)

while( sel_pointID != 'Point_EP' ) : 
    #print("시작포인트 : " , point)
    points = pd.DataFrame()
    points = get_points_in_range(df, point)    #1. 반경 내 속하는 점들 검색
    points['F'] = get_f(points)                #2. F=G+H : F구하기
    points['scaled_F'] = get_scaled_f(point, points)  #3. F를 밝기에 맞춰 스케일링
    points['W'] = get_w(points)                #4. 최종가중치 구하기
    
    points = points.sort_values('W', ascending = False).reset_index() #5. 정렬

    print(points.loc[0,'point_ID'])
    sel_pointID = points.loc[0,'point_ID']
    point = (points.loc[0,'위도'] , points.loc[0,'경도'])

    #goodbye.append(points.loc[1:,'point_ID'].values)
    heapq.heappush(closed_list_id, sel_pointID)
    heapq.heappush(closed_list_seq, n)
    heapq.heappush(closed_list, ( points.loc[0,'위도'] , points.loc[0,'경도']))
    n+=1
   # print(sel_pointID)

Point_1789
Point_1379
Point_1316
Point_1373
Point_1372
Point_1317
Point_1382
Point_1380
Point_1381
Point_2072
Point_1298
Point_1296
Point_1378
Point_1292
Point_1293
Point_1295
Point_1805
Point_1374
Point_1321
Point_1834
Point_1503
Point_1320
Point_1371
Point_1318
Point_1323
Point_1322
Point_1319
Point_1437
Point_1502
Point_1438
Point_703
Point_2023
Point_1377
Point_704
Point_1504
Point_1445
Point_1375
Point_1376
Point_1439
Point_1637
Point_1385
Point_1326
Point_1297
Point_1294
Point_1892
Point_1722
Point_167
Point_278
Point_200
Point_279
Point_169
Point_173
Point_172
Point_2005
Point_247
Point_248
Point_253
Point_EP


In [206]:
# goodbye = pd.DataFrame(columns = ['위도', '경도'] )
# goodbye.append(points.loc[1:,'point_ID'].values)
points.loc[1:,'point_ID'].values
closed_list_id[57]

'Point_253'

In [207]:
# 역삼동 중심 = 역삼역 /  좌표 : 37.500742, 127.036891
to_map1 = folium.Map(location= sp, zoom_start=18)
a = closed_list
b 
# for n in range(0, len(a)) :|
#     if a['분류'][n] == '편의점' :
#         folium.Marker([a['위도'][n], a['경도'][n]], popup= a['분류'][n], icon=folium.Icon(color='green')).add_to(to_map1)
#     elif a['분류'][n] == 'CCTV' : 
#         folium.CircleMarker([a['위도'][n], a['경도'][n]], popup= a['point_ID'][n] + a['명칭'][n], radius = a['밝기'][n] + 1).add_to(to_map1)        
#     elif a['분류'][n] == '보안등' : 
#         folium.CircleMarker([a['위도'][n], a['경도'][n]], popup= a['point_ID'][n] + a['명칭'][n], radius = a['밝기'][n] + 1).add_to(to_map1)  
#     else : 
#         folium.Marker([a['위도'][n], a['경도'][n]], popup= a['분류'][n], icon=folium.Icon(color='black')).add_to(to_map1)        


for n in range(0, len(a)) :
      folium.CircleMarker([a[n][0], a[n][1]], popup= str(closed_list_seq[n]), radius = 3).add_to(to_map1)        

# 구분하기 쉽게 스타팅 지역은 빨간색 마커로 표시        
folium.Marker(sp, popup= 'SP', icon=folium.Icon(color='red')).add_to(to_map1)
folium.Marker(ep, popup= 'EP', icon=folium.Icon(color='black')).add_to(to_map1)
to_map1


In [208]:
df[df['point_ID'] == 'Point_248' ]
df[df['명칭'].str.contains('목화') ]

#closed_list_id

,ID,point_ID,경도,명칭,밝기,분류,위도,총밝기
1717,1795,Point_1795,127.043,GS25 역삼목화점,30,편의점,37.5089,75


In [211]:
dd = total_df[ total_df['point_ID'].isin(closed_list_id) ].sort_values('총밝기', ascending = False)
dd

,Unnamed: 0,point_ID,거리,밝기,총밝기,분류,명칭,위도,경도,경유여부
1789,1789,Point_1789,0,30,110,편의점,CU 역삼에스원점,37.506208,127.037951,0
1834,1834,Point_1834,0,25,110,CCTV,역삼동 655-13,37.506711,127.037181,0
1437,1437,Point_1437,0,5,110,보안등,역삼1동 661-3,37.505838,127.038051,0
1379,1379,Point_1379,0,5,105,보안등,역삼1동 656-12,37.506415,127.037943,0
1323,1323,Point_1323,0,5,105,보안등,역삼1동 655-13,37.506913,127.037214,0
1438,1438,Point_1438,0,5,105,보안등,역삼1동 660-33,37.505781,127.037757,0
1637,1637,Point_1637,0,30,100,편의점,GS25 역삼충현점,37.505715,127.037723,0
1316,1316,Point_1316,0,5,90,보안등,역삼1동 656-5,37.506599,127.037603,0
1503,1503,Point_1503,0,5,85,보안등,역삼1동 661-1,37.506062,127.037942,0
1317,1317,Point_1317,0,5,85,보안등,역삼1동 657-40,37.506480,127.037314,0


In [210]:
df[df['point_ID'] == 'Point_253' ]

,ID,point_ID,경도,명칭,밝기,분류,위도,총밝기
227,253,Point_253,127.043,역삼1동 684-27,5,보안등,37.5089,75
